In [ ]:
import pandas as pd
import requests
import csv

html = requests.get("http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20181205&type=ALLBUT0999&_=1544171814819")

with open("stock.csv","w",encoding="utf-8-sig") as fp:
    fp.write(html.text)
with open("stock.csv","r",encoding="utf-8-sig") as fp:
    content = list(csv.reader(fp))
    
# 找出content裡有17個欄位的元素，加到stockLi裡
stockLi = []
for i in content:
    if len(i) == 17:
        del i[-1]
        stockLi.append(i)
        
# 刪除第 0 列
del stockLi[0]

# 整理資料裡無用的符號，字串的replace()會成為新的元素，所以要重新寫回
for i in range(len(stockLi)):
    stockLi[i][0] = stockLi[i][0].replace('"','')
    stockLi[i][0] = stockLi[i][0].replace('=','')
    for j in range(2,16):
        stockLi[i][j] = stockLi[i][j].replace(',','')
        
# 將有多個 List的 List直接轉成 DataFrame
from pandas.core.frame import DataFrame

stockDf = pd.DataFrame(stockLi[1:],columns= stockLi[0])
stockDf = stockDf.set_index("證券代號")
stockDf
# 找出有開盤且收益大於 1.05 的股票
# for i in range(1,len(stockLi)):
#     if stockLi[i][5] != '--' and (float(stockLi[i][8])/float(stockLi[i][5]))>1.05:
#         print(stockLi[i])

In [ ]:
import sqlite3

conn = sqlite3.connect("stock.sqlite3")

stockDf.to_sql("data20181213",conn)
stockSql=pd.read_sql("select * from data20181213",conn,index_col=["證券代號"])
print (stockSql.head())